# Bib for external:

In [6]:
%run function.py

In [ ]:
import potentialfinder



In [ ]:
potentialfinder.find_exponents.__doc__

# Examples

## loading testdata in a dataframe

In [ ]:
#sqlDFTest = spark.sql("SELECT * FROM db_prepared_bdas_rsu_local_e2e.bt_tal_main limit 10")
#sqlDF.show(1)
#sqlDFTest = spark.sql("select * from db_source_gwk_local.v_bdgwk_t_gw_antrep_gut where rand() <= 0.0001 distribute by rand() sort by rand() limit 1000")
#sqlDFTest=pd.read_csv('kagglefiles\\aashita\\nyt-comments\\CommentsMay2017.csv')
sqlDFTest=pd.read_csv('db_source_gwk_local.v_bdgwk_t_gw_antrep_gut.csv',sep=";")

sqlDFTest.head(2)
resulttable=find_exponents(sqlDFTest,
                           fractionToAnalyze=1,
                           outputPath='/outputs',
                           outputTable=True,
                           outputPlots=True,
                           outputTablename='Testtable',
                           logToScreen=True,
                           columnFillThreshold=0.5,
                           exp_threshold=0.1,
                           r_s_threshold=0.7,
                           maxrows=5000,
                           debug=False)


resulttable


## Run the function on the test data:

In [ ]:
resulttable=find_exponents(sqlDFTest,
                           fractionToAnalyze=1,
                           outputPath='/outputs',
                           outputTable=True,
                           outputPlots=True,
                           outputTablename='Testtable',
                           logToScreen=True,
                           columnFillThreshold=0.5,
                           exp_threshold=0.1,
                           r_s_threshold=0.7,
                           maxrows=5000,
                           debug=False)


resulttable



## Run multiple queries with different parameters but add them up to one result dataframe

In [ ]:
firstrow=True
for fraction in [1,0.5,0.25]:
    temptable=find_exponents(sqlDFTest,
                           fractionToAnalyze=fraction,
                           outputPath='outputs',
                           outputTable=False,
                           outputPlots=False,
                           outputTablename='Testtable',
                           logToScreen=False,
                           columnFillThreshold=0.5,
                           exp_threshold=0.6,
                           r_s_threshold=0.7,
                           maxrows=5000)
    if firstrow:
        resulttable=temptable
        firstrow=False
    else:
        resulttable=resulttable.append(temptable)
        
resulttable
    

In [ ]:
resulttable

## Scientific graphs for a subset

In [1]:
###get a list of all authors to convert (in the selected folder)
from os import listdir
path='selected'
folderlist=[]
authorlist=[]
for filename in listdir(path):

    print(filename)
    authorname=filename.split('_')[1]
    foldername=filename.split('_')[2]
    if not foldername in folderlist:
        folderlist.append(foldername)
    if not authorname in authorlist: authorlist.append(authorname)

folderlist

kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-100perc-log.png
kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-100perc-nonlog.png
kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-25perc-log.png
kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-25perc-nonlog.png
kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-50perc-log.png
kagglefiles_azathoth42_myanimelist_users_cleaned.csv-join_date-user_id-50perc-nonlog.png
kagglefiles_census_total-construction-spending-data-collection_total-construction-spending-residential_1.csv-date-value-25perc-log.png
kagglefiles_census_total-construction-spending-data-collection_total-construction-spending-residential_1.csv-date-value-25perc-nonlog.png
kagglefiles_census_total-construction-spending-data-collection_total-construction-spending-residential_1.csv-date-value-50perc-log.png
kagglefiles_census_total-construction-spending-data-collection_to

['myanimelist',
 'total-construction-spending-data-collection',
 'total-merchant-wholesalers-inventory-and-sales',
 'chicago-lobbyist-data',
 'chicago-sidewalk-cafe-permits',
 'los-angeles-metro-bike-share-trip-data',
 'oakland-call-center-public-work-service-requests',
 'ny-bus-breakdown-and-delays',
 'nba-enhanced-stats',
 'mlb-pitch-data-20152018',
 'stock-time-series-20050101-to-20171231',
 'chicago-divvy-bicycle-sharing-data']

In [17]:
authorlist

['azathoth42',
 'census',
 'chicago',
 'cityofLA',
 'cityofoakland',
 'new-york-city',
 'pablote',
 'pschale',
 'szrlee',
 'yingwurenjian']

In [4]:
%run function.py
global_x=[]
from os import listdir
from os.path import isfile,isdir, join
import codecs
import cchardet
import random
from io import StringIO
import csv
#import potentialfinder
import pandas as pd




def get_encoding(file,skiprows=0):
    testStr = b''
    count = 0
    with open(file, 'rb') as x:
        line = x.readline()
        while line and count < 5000:  #Set based on lines you'd want to check
            for _ in range(0,skiprows): #read in only those rows which are later used for sampling as well 
                try:
                    next(x)
                except StopIteration:
                    pass
            testStr = testStr + line
            count = count + 1
            line = x.readline()
    result=cchardet.detect(testStr)
    if result['confidence'] is None:
        return ''
    if result['confidence']<1:
        print('# Encoding Detection: ' + result['encoding']+' (confidence:' + str(result['confidence'])+")")
    return result['encoding']

def buffered_row_count(file):
    count = 0
    thefile = open(file, 'rb')
    while 1:
        buffer = thefile.read(8192*1024)
        if not buffer: break
        count += buffer.count(b'\n')
    thefile.close(  )
    return count

def read_every_nth_row_to_byte(file,n_th):
    ## required as pandas.read_csv with the skip function takes too much memory. 
    ## --> first load every nth row as byte --> then load this into pandas
    with open(file, 'rb') as x:
        byteStr = b''
        line = x.readline()
        #line = line.replace(b'\n', b'')+b'\n'
        #print(line)
        while line:  #Set based on lines you'd want to check
            for _ in range(0,n_th): #read in only those rows which are later used for sampling as well 
                try:
                    next(x)
                except StopIteration:
                    pass
            byteStr = byteStr + line
            line = x.readline()
            #line = line.replace(b'\n', b'')+b'\n'
            #print(line)
        byteStr = byteStr + line
    return byteStr


def get_last_analyzed_file(analyzed_files):
    if isfile(analyzed_files):
        analyzed_files_obj = open(analyzed_files, "r")
        allfiles=analyzed_files_obj.read().splitlines() 
        analyzed_files_obj.close()
        
        return allfiles[-1]
    else:
        return ''

def update_analyzed_files(file_to_add,analyzed_files):
    analyzed_files_obj = open(analyzed_files, "a+")
    analyzed_files_obj.write(file_to_add+'\n')
    analyzed_files_obj.close()

n_rows=10000




path="kagglefiles"
debug=True
analyzed_files="analyzed_files.txt"
outputtablefile='outputtable_selected.csv'
skip_files=True
##read list of files that have already been analyzed (delete this file to force restart analysis on all files):
last_analyzed_file=get_last_analyzed_file(analyzed_files)
print("Last analyzed File: "+last_analyzed_file)

if last_analyzed_file=="":
    firstrow=True
else:
    firstrow=False
    resulttable=pd.read_csv("outputs_selected/table/"+outputtablefile,sep=";")
    

for username in listdir(path):
    

    if True:
    #if username in authorlist:
        if isdir(join(path, username)):
            for datatablename in listdir(join(path, username)):
                if True:
                #if datatablename in folderlist:
                    if isdir(join(path, username,datatablename)):
                        for filename in listdir(join(path, username,datatablename)):
                            folder_and_file=join(path,username,datatablename,filename)
                            if isfile(folder_and_file):
                                if filename[-3:]!='csv':
                                    continue
                                outputTableName=path+'_'+username+'_'+datatablename+'_'+filename
                                if path+username+datatablename+filename==last_analyzed_file.replace('\\','') or last_analyzed_file=='':
                                    skip_files=False
                                if skip_files:
                                    print("#Skipping:"+ folder_and_file)
                                    continue
                                print("#Computing:"+ folder_and_file)
                                update_analyzed_files(folder_and_file,analyzed_files)
                                try:
                                    
                                    num_lines = buffered_row_count(folder_and_file) #requires no encoding 
                                    if num_lines<=n_rows:
                                        n_th=0
                                    else:
                                        n_th = round(num_lines/n_rows)  # every 100th line = 1% of the lines
                                    if num_lines == 0:
                                        print(folder_and_file + " seems empty.")
                                        continue
                                    encoding=get_encoding(folder_and_file,n_th)
                                    print("-> Encoding:"+encoding)
                                    try:
                                        if encoding=='':
                                            datastr=str(read_every_nth_row_to_byte(folder_and_file,n_th))
                                        else:
                                            datastr=str(read_every_nth_row_to_byte(folder_and_file,n_th),encoding=encoding)
                                    except:
                                        print("Skipped because of encoding identification error.")
                                        continue
                                    
                                    
                                    separators=[',',';','|','\t']
                                    quotechars=['"',"'"]
                                    worked=False
                                    for separator in separators:
                                        for quotechar in quotechars:
                                            if not worked:
                                                try:
                                                    data=StringIO(datastr)
                                                    print("trying " + separator + " | " + quotechar)
                                                    sqlDFTemp = pd.read_csv(data, sep = separator,encoding=encoding, quotechar=quotechar)#, error_bad_lines=False)
                                                    if len(sqlDFTemp.columns)==1:
                                                        raise ###separator has not worked out if there is only 1 column
                                                    worked=True
                                                except:
                                                    try:
                                                        data=StringIO(datastr)
                                                        print("trying " + separator + " | " + quotechar)
                                                        sqlDFTemp = pd.read_csv(data, sep = None,encoding=encoding, engine = 'python', error_bad_lines=False)
                                                        worked=True
                                                    except:
                                                        1==1
                                    if worked==False:
                                        print("skipped")
                                        raise
                                        continue
                                    
                                        #sqlDFTemp=pd.read_csv(join(path,username,datatablename,filename), sep = None, engine = 'python')
                                except UnicodeDecodeError:
                                    print ("Unicode Decode Error")
                                except:
                                    if debug:
                                        raise
                                    else:
                                        print('error parsing file: '+join(path,username,datatablename,filename))
                                        continue
                                
                                for fraction in [1,0.5,0.25]:

                                    #temptable=potentialfinder.find_exponents.find_exponents(sqlDFTemp,
                                    temptable=find_exponents(sqlDFTemp,
                                                            fractionToAnalyze=fraction,
                                                            outputPath='outputs_selected3',
                                                            outputTable=True,
                                                            outputPlots=True,
                                                            outputTablename=outputTableName,
                                                            logToScreen=False,
                                                            columnFillThreshold=0.5,
                                                            exp_b_threshold=0.00000000002,
                                                            exp_r_s_threshold=0.7,
                                                            maxrows=n_rows,
                                                            debug=False)
                                    if firstrow:
                                        resulttable=temptable
                                        firstrow=False
                                    else:
                                        if not temptable is None:
                                            resulttable=resulttable.append(temptable,ignore_index=True)
                                            try:
                                                resulttable.to_csv("outputs_selected3/table/"+outputtablefile,sep=";")
                                            except:
                                                print("Writing error. Resuming.")
                                
#resulttable
    

9772    3.317837e-08
9707    3.329478e-08
9791    3.329478e-08
9698    3.329478e-08
Name: date_days_since_first_date, Length: 2494, dtype: float64
236     2.502929e-08
213     2.502929e-08
262     2.502929e-08
309     2.502929e-08
289     2.502929e-08
            ...     
9550    3.317837e-08
9772    3.317837e-08
9707    3.329478e-08
9791    3.329478e-08
9698    3.329478e-08
Name: date_days_since_first_date, Length: 2494, dtype: float64
236     2.502929e-08
213     2.502929e-08
262     2.502929e-08
309     2.502929e-08
289     2.502929e-08
            ...     
9550    3.317837e-08
9772    3.317837e-08
9707    3.329478e-08
9791    3.329478e-08
9698    3.329478e-08
Name: date_days_since_first_date, Length: 2494, dtype: float64
213     2.502929e-08
432     2.502929e-08
237     2.502929e-08
289     2.502929e-08
236     2.502929e-08
            ...     
9550    3.317837e-08
9772    3.329478e-08
9707    3.329478e-08
9698    3.329478e-08
9791    3.329478e-08
Name: date_days_since_first_date, 

In [ ]:
resulttable.to_csv("outputs\\table\\outputtable.csv",sep=";")